In [1]:
import re

import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By

from fake_useragent import UserAgent
from fake_useragent import FakeUserAgentError
from math import ceil
from random import randint
from time import sleep
from base64 import b64decode
from io import BytesIO
from PIL import Image
import pandas as pd
from tqdm import tqdm

In [2]:
#URL = "https://www.avito.ru/all/avtomobili?f=ASgBAgICAUTyCrCKAQ" #не битые авто
URL = "https://www.avito.ru/all/avtomobili?f=ASgBAgECAUTyCrCKAQFF~owUF3siZnJvbSI6bnVsbCwidG8iOjIwMjB9"#последние объявления до 2020 года

In [3]:
def avito_parser(i):
    try:
        ua = UserAgent().chrome
    except FakeUserAgentError:
        ua = "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:84.0) Gecko/20100101 Firefox/84.0"
    options = webdriver.ChromeOptions()
    options.add_argument(f"user-agent={ua}")
    options.add_argument("--disable-blink-features=AutomationControlled")

    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(10)
    driver.get(URL)

    #count = int(
    #    (
    #        driver.find_element(
    #            by=By.CSS_SELECTOR, value='span[data-marker="page-title/count"]'
    #        ).text
    #    ).replace(" ", "")
    #)

    popular_models = driver.find_element(
            by=By.CSS_SELECTOR, value='div[data-marker="popular-rubricator/links"]'
        ).find_elements(
            by=By.CSS_SELECTOR, value='a[data-marker="popular-rubricator/link"]'
        )
    
    #for i in popular_models:
    #    print(i.text)

    df = pd.DataFrame(columns=['name', 'price', 'params', 'desc', 'region'])

    for _ in range(1):
        model = popular_models[i]
    #for i in tqdm(range(20)):
        #print(model.text)
        url_model = model.get_attribute('href')
        driver.get(url_model)

        for _ in tqdm(range(20)):

            offers = driver.find_elements(
                by=By.CSS_SELECTOR, value='div[data-marker="item"]'
            )

            for offer in offers:
                try:
                    #print(offer.text)
                    avito_id = int(offer.get_attribute("id")[1:])

                    #offer_text = offer.text.split('\n')

                    name = offer.find_element(
                        by=By.CSS_SELECTOR, value='h3[itemprop="name"]'
                    ).text

                    #name = offer_text[0]

                    #print(offer[0])
                    #print(offer[1])
                    #print(offer[2])

                    price = offer.find_element(
                        by=By.CSS_SELECTOR, value='meta[itemprop="price"]'
                    ).get_attribute("content")

                    #price = offer_text[1]

                    params = offer.find_element(
                        by=By.CSS_SELECTOR, value='p[data-marker="item-specific-params"]'
                    ).text

                    #params = re.search(r'([^,\n]+,){4}[^,\n]+', offer.text, re.IGNORECASE)[0]

                    desc = offer.find_element(
                        by=By.CSS_SELECTOR, value='div[class="iva-item-descriptionStep-C0ty1"]'
                    ).text
                    #desc = offer_text[-3]

                    region = offer.find_element(
                        by=By.CSS_SELECTOR, value='div[class="geo-root-zPwRk"]'
                    ).text
                    #region = offer_text[-2]

                    #print(f"name: {name}")
                    #print(f"price: {price}")
                    #print(f"params: {params}")
                    #print(f"desc: {desc}")
                    #print(f"region: {region}")
                    df.loc[avito_id, 'name':'region'] = [name, price, params, desc, region]

                except Exception as ex:
                    print(ex)

            try:    
                driver.find_element(
                    by=By.CSS_SELECTOR, value='a[data-marker="pagination-button/nextPage"]'
                ).click()

                print(f'len_df: {len(df)}')

                rand_sleep = randint(25, 49)
                sleep(rand_sleep / 10)

            except Exception as ex:
                    print(ex)

        df.to_csv(f'./datasets2/data{i}.csv')


    driver.quit()

    return df

In [ ]:
i = [0, 1, 3, 6, 7, 8, 9, 12, 13, 14, 15, 16, 18, 19, 22, 23, 25]
for k in i:
    print(k)
    a = avito_parser(k)